## COPMAS Replication Study ##

*Lab adapted from [the ProPublica analysis](https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb) and an exercise by [Darakhshan Mir](http://eg.bucknell.edu/~djm056/)*

In this lab, we will build our own COMPAS-like predictor using logistic regression and the data provided by ProPublica. Here we're following [the methodology documented by ProPublica](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm). 

### Getting Started

First let's load the libraries we'll need:

In [ ]:
# Making sure we have all the right libraries
%matplotlib inline

import pandas as pd
import pylab
import numpy as np
import matplotlib.pyplot as plt

Now let's load the data:

In [ ]:
data = pd.read_csv('data/compas-scores-two-years.csv', index_col=0)

data.shape

**What does this tell us?**

In [ ]:
# take a look at the top 

pd.options.display.max_columns = None # have to do this otherwise it limits the number of cols shown

data.head() 

**What's a function that gives us the basic info on a pandas dataset?**

**What do we note here in these counts?**

Because some data are missing, not all the rows are usable for the first round of analysis.

Unlike the Titanic exercise, we're not going to interpolate our missing values. Here, we're just going remove the ones we can't use. More specifically: 

First we'll filter out those which do not have a COMPAS-scored case, as indicated by the recidivist flag `is_recid` set at -1.

In [ ]:
# Filtering data
filterData = data[(data['is_recid'] != -1)]

filterData.shape

Within the cases with a COMPAS score, we also need to check to see if we have the right offense. So if the charge date of a defendant's COMPAS-scored crime was not within 30 days from when the person was arrested, it's best to assume that we do not have the right offense, and remove that row.

So we will filter out rows where **days_b_screening_arrest** is over 30 or under -30:

In [ ]:
# Filtering data
filterData = data[(data['days_b_screening_arrest'] <= 30) & (data['days_b_screening_arrest'] >= -30)]

In [ ]:
filterData.shape

In [ ]:
# Let's just take a quick look at some of the demographics:

In [ ]:
filterData.age_cat.value_counts()

In [ ]:
filterData.race.value_counts()

In [ ]:
# What about how you look at normalized value counts?



**What does this tell us about the racial composition of the dataset?**

In [ ]:
# Now let's look at the risk scores

filterData.score_text.value_counts()

Here is another useful pandas function: **crosstab**. Let's see what it does:

In [ ]:
# recidivsm rates by race
pd.crosstab(filterData.sex, filterData.race)

**Can you find out how many men and how many women are in the dataset overall?**

In [ ]:
# remove
filterData.sex.value_counts()

**What about the percentage of men and women in the dataset?**

We've already looked at the risk scores. But in fact, judges are often presented with two sets of scores from the Compas system -- one that classifies people into High, Medium and Low risk, and a corresponding decile score. ProPublica found that there is a clear downward trend in the decile scores as those scores increase for white defendants, as shown below.

In [ ]:
scores_by_race = pd.crosstab(filterData.race, filterData.decile_score)

scores_by_race

In [ ]:
# same thing visualized:

labels = list(scores_by_race.columns)

aa_scores = list(scores_by_race.loc["African-American"])
c_scores = list(scores_by_race.loc["Caucasian"])

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, aa_scores, width, label='African-American')
rects2 = ax.bar(x + width/2, c_scores, width, label='Caucasian')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Count')
ax.set_title('Scores by decile and race')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.show()


## Racial Bias in COMPAS ##

The chart above suggests that *something* is going on. But in order to test our intution that there is a significant difference in COMPAS scores across different racial categories, we need to change some variables into factors and run a logistic regression, comparing low scores to high scores.

The first step is to convert the c_charge_degree, age_cat, race, sex (which are all categorical data) into factors. 

*Switch to [ProPublica audit](https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb)*
